# **Predicting Tacos Notes**

***
The goal of this project is to analyze food data. We will use a dataset about burritos. It comes with feature characteristics about the customers, composition of the burrito, locations *etc.*. The dataset is available on GoogleSheets and can be downloaded there at a csv format before uploading it on DataBricks:
https://docs.google.com/spreadsheets/d/18HkrklYz1bKpDLeL-kaMrGjAhUM6LeJMIACwEljCgaw/edit#gid=1703829449


First of all the aim will be to predict the note given by users to burritos they ate. It involves building a model to do so given explanatory variables originally implemented in the dataset but also with new ones created while doing data preparation. Besides, we want to do this project along with a big data framework. Hence, treatments will be done using Spark coding language.

In a second part the aim will be to predict a convenient burrito composition given user information. Such techniques could be used to customize commercials sent to customers by email ads or on online food delivering platforms. 
***

## Packages

In [3]:
#pyspark.sql functions: when, count, col etc.
from pyspark.sql import functions as fn
from pyspark.sql import Window

#ml lib tools:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, Tokenizer, StopWordsRemover
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier
from pyspark.ml.regression import LinearRegression

from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.stat import Correlation
from pyspark.mllib.stat import Statistics
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorIndexer, IndexToString

#Array, mathematics operations:
import numpy as np
import matplotlib.pyplot as plt

#Dataframes:
import pandas as pd

#Sentiment analysis:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## 1 Data Import

In [5]:
#file path
df_path = '/FileStore/tables/Burrito.csv'

#Loading the Burrito csv file
df = spark.read.format('csv').options(header='true', inferSchema='true').load(df_path)

Let's have a look at the whole database:

In [7]:
display(df)

Location,Burrito,Date,Neighborhood,Address,URL,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa22,Synergy,Wrap,overall,Rec,Reviewer,Notes,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini
Donato's taco shop,California,1/18/2016,Miramar,6780 Miramar Rd,http://donatostacoshop.net/,3.5,4.2,null,6.49,3.0,null,null,null,null,null,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,3.8,null,Scott,good fries: 4/5,null,null,x,x,x,x,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Oscar's Mexican food,California,1/24/2016,San Marcos,225 S Rancho Santa Fe Rd,http://www.yelp.com/biz/oscars-mexican-food-san-marcos,3.5,3.3,null,5.45,3.5,null,null,null,null,null,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,3.0,null,Scott,Fries: 3/5; too little meat,null,null,x,x,x,x,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Oscar's Mexican food,Carnitas,1/24/2016,null,null,null,null,null,null,4.85,1.5,null,null,null,null,null,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,3.0,null,Emily,null,null,null,null,x,x,null,null,null,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Oscar's Mexican food,Carne asada,1/24/2016,null,null,null,null,null,null,5.25,2.0,null,null,null,null,null,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,3.75,null,Ricardo,Go to average burrito place like Rigoberto's in la jolla; Terrible tamales,null,null,x,x,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Pollos Maria,California,1/27/2016,Carlsbad,3055 Harding St,http://pollosmaria.com/,4.0,3.8,x,6.59,4.0,null,null,null,null,null,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,4.2,null,Scott,null,null,null,x,x,null,x,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Pollos Maria,combo chicken,1/28/2016,null,null,null,null,null,null,6.99,4.0,null,null,null,null,null,3.0,4.0,5.0,3.5,2.5,2.5,2.5,4.0,1.0,3.2,null,Emily,null,null,null,null,null,x,x,null,x,null,x,null,null,x,x,x,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Nico's Taco Shop,California,1/30/2016,Carmel Valley,3860 Valley Centre Dr #404,http://www.yelp.com/biz/nicos-taco-shop-san-diego,3.0,2.9,null,7.19,1.5,null,null,null,null,null,2.0,3.0,3.0,2.0,2.5,2.5,null,2.0,3.0,2.6,null,Scott,not fries. big potatoes,null,null,x,null,null,x,x,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Nico's Taco Shop,Carnitas,1/30/2016,null,null,null,null,null,null,6.99,4.0,null,null,null,null,null,2.5,3.0,3.0,2.5,3.0,3.5,null,2.5,3.0,3.0,null,Emily,null,null,null,null,x,x,null,null,null,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Los Primos Mexican Food,Monster California,2/1/2016,UTC,7770 Regents Rd,http://www.primosmex.com/,3.0,3.7,x,9.25,3.5,null,null,null,null,null,2.0,4.5,4.5,3.5,1.5,3.0,3.5,4.0,2.0,3.9,null,Scott,this tasted really bad leftover. not included in rating,null,null,x,x,x,x,x,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
JV's Mexican Food,Carne Asada,2/6/2016,Morena,1112 Morena Blvd,http://jvsmexfood.com/,4.0,4.1,null,6.25,3.5,n

#### Dataset dimensions

In [9]:
#Dataset dimensions:
N = df.count() #N observations
P = len(df.columns) #P columns
print(f'There are {N} observations and {P} columns in the dataset.')

There are 423 observations and 66 columns in the dataset.

## 2 Data Cleaning

### 2.1 Observations ID

The dataset does not contain the id for each observation, we can do so as followed:

In [13]:
df = df.withColumn('id', fn.monotonically_increasing_id())

### 2.2 Handling missing values

Before computing preliminary descriptive statistics and reformatting variable, let's display the number of missing values per column:

In [16]:
na_per_column = df.select([fn.count(fn.when(df[c].isNull(), c)).alias(c) for c in df.columns])
display(na_per_column)

Location,Burrito,Date,Neighborhood,Address,URL,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa22,Synergy,Wrap,overall,Rec,Reviewer,Notes,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,id
0,0,0,331,335,336,336,336,397,7,3,401,401,139,141,141,0,20,14,3,9,2,25,2,3,2,190,1,277,390,416,243,264,268,263,295,331,372,402,402,417,387,388,412,416,416,422,415,385,416,408,406,419,416,421,419,419,422,423,418,420,420,421,410,420,422,0


For the rest of the project we have to keep in mind that the missing value format is expressed as 'null', hence isNull() or isNotNull() functions will be used instead of isna().

#### 2.2.1 `Neighborhood`, `Adress`, `Yelp` & `Google`

We noticed the columns `Neighborhood`, `Adress`, `Yelp`, `Google` only contain value for the first time a shop is rated in the dataset. We want to fill the NAs according to the `Location`, which is the **join key**. Hence we will filter the dataset accross distinct non null values then merge both data frames according to the `Location`.

In [20]:
for c in ['Neighborhood', 'Yelp', 'Google']:
  #Create a temporary dataframe where only distinct non value for 'Location' and c are stored:
  temp = df.select("Location", c).distinct()
  
  #filtering on non null values:
  temp = temp.filter(temp[c].isNotNull())
  
  #Merging with the original dataset df:
  df = df.drop(c).join(temp,'Location', 'left')
  
#Display the results:
display(df.limit(5))

Location,Burrito,Date,Address,URL,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa22,Synergy,Wrap,overall,Rec,Reviewer,Notes,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,id,Neighborhood,Yelp,Google
Lolita's Taco shop,Carne asada,5/27/2016,null,null,null,6.25,4.0,null,null,15.0,20.5,0.5,1.5,2.5,3.0,4.5,4.0,2.0,2.75,3.75,3.0,2.6,No,Scott,Way too small,null,null,X,X,X,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,120,null,null,null
Lolita's Taco shop,Shrimp,5/27/2016,null,null,null,8.95,3.5,null,null,18.5,25.0,0.92,3.0,3.0,4.0,4.0,3.5,3.5,2.5,4.0,0.5,3.0,Yes,Emily,null,null,null,null,X,null,null,null,null,null,null,X,null,null,null,X,null,null,null,null,X,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,121,null,null,null
Lolita's Taco shop,Bean and cheese,5/27/2016,null,null,null,3.5,1.0,null,null,16.0,20.0,0.51,3.5,4.5,null,3.0,null,4.0,3.0,2.5,4.0,2.7,No,Scott,null,null,null,null,null,null,x,null,null,null,null,null,null,null,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,122,null,null,null
Tacos La Bala,Pastor,9/5/2016,5800 Bellaire Blvd,tacoslabala.com,null,4.99,3.5,null,null,null,null,null,2.5,2.5,4.0,2.5,1.0,1.0,2.0,4.0,0.0,2.5,No,Scott,Everything is in spanish. Get a taco or sope,null,X,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,222,Houston,4.5,4.1
Tacos La Bala,Barbacoa,9/5/2016,null,null,null,4.99,2.5,null,null,16.5,22.0,0.64,2.0,2.5,3.5,2.0,3.0,2.5,2.0,2.5,2.5,2.4,No,Emily,"Burritos cut in half, felt like a wrap instead of a burrito",null,X,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,223,Houston,4.5,4.1


Let's check if there are still missing values:

In [22]:
display(df.select([fn.count(fn.when(df[c].isNull(), c)).alias(c) for c in ['Neighborhood', 'Yelp', 'Google']]))

Neighborhood,Yelp,Google
63,66,66


There are still missing values, but we will handle them by setting them into a class when we will do the data preparation.

#### 2.2.2 `Cost` & `Hunger`

These two variables do not contain so much NAs, hence we can replace them by imputing their mean value.

For both columns, let's compute the mean over the column values not containing any null:

1. We have to first filter over df where it's column `Cost` isn't null
2. Compute the average
3. Collect the value with the first() operation and retain it by adding [0] to keep the value within it
4. use numpy.round() function to retain only 2 decimals

In [26]:
for c in ['Cost', 'Hunger']:
  #Calculating the mean and round it to 2 decimals:
  mean = np.round(df.where(df[c].isNotNull()).agg({c : 'avg'}).first()[0],2)
  print(f"The average value for column '{c}' is about {mean}")

  #Create the decision rule; according to a null value condition either we inpute the value either the computed mean
  new = fn.when(df[c].isNull(), mean)\
               .otherwise(df[c])

  #We apply the decision rule and inpute it to actual column:
  df = df.withColumn(c, new)

The average value for column 'Cost' is about 7.07
The average value for column 'Hunger' is about 3.5

#### 2.2.3 Burrito's composition dummies

There are several columns indicating whether the burritos contains a food or not. As they are filled with "x" indicating the food is present, or "null" indicating the food is absent, we will display those variables in a better format. The column `Chips` is located apart from the others so we will treat it right after.

In [29]:
#Select the food variable names
columns_food = df.columns[28:63]

#Display:
display(df.select(columns_food).limit(5))

Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini
X,X,X,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,X,null,null,null,null,null,null,X,null,null,null,X,null,null,null,null,X,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,x,null,null,null,null,null,null,null,x,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


We can do a loop over those variables in order to re-set them in a good format:

In [31]:
#loop over the columns in `columns_food`:
for c in columns_food:
  #Decision rule: 0 if null, 1 otherwise
  binary = fn.when(df[c].isNull(), 0)\
             .otherwise(1)
  
  #update the corresponding column:
  df = df.withColumn(c,binary)

Let's do it on `Chips`:

In [33]:
Chips = fn.when(df['Chips'].isNull(), 0)\
          .otherwise(1)
df = df.withColumn('Chips', Chips)

Checking it has been done correctly:

In [35]:
#Display:
display(df.select(columns_food).limit(5))

Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### 2.2.4 `Notes`

Some notes are missing, but it is compulsory to have a string value in order to do text or sentiment analysis on it:

In [38]:
#Decision rule, we set "Neutral" whenever the value is missing:
notes = fn.when(df["Notes"].isNotNull(), df["Notes"])\
          .otherwise("Neutral")

#We apply it:
df = df.withColumn("Notes", notes)

#Display of 5 rows:
df.select("Notes").show(5)

+--------------------+
 Notes|
+--------------------+
 Way too small|
 Neutral|
 Neutral|
Everything is in ...|
Burritos cut in h...|
+--------------------+
only showing top 5 rows

#### 2.2.5 `overall`

`overall`, the target variable, contains 2 missing, values, as we can't know the real values for them, we can discard the corresponding rows and reset the number of observations.

In [41]:
#Discarding rows for overall is missing:
df = df.filter(df['overall'].isNotNull())

#Reset the number of observations:
N = df.count() #N observations
print(f'There are {N} observations.')

There are 421 observations.

### 2.3 Dropping variables

As we noticed that `Neighborhood`,`URL` and `Adress` were filled the same way in the dataset, we do not want repetitive information within it, so we can delete those columns.

Furthermore, we can delete it **endogenous variables** such like variables linked to the dependent variable `overall` itself: `Tortilla`, `Meat`, `Fillings`, `Meat:filling`, `Uniformity`, `Salsa22`, `Synergy`, `Wrap`

Other features such as `Rec`, `Date`, `Temp` and `Address` might also be discarded:

In [44]:
df = df.drop('URL','Adress', 'Mass (g)', 'Density (g/ML)', 'Unreliable', 'NonSD', 'Tortilla', 'Meat',
             'Fillings', 'Meat:filling', 'Uniformity', 'Salsa22', 'Synergy', 'Wrap', 'Rec', 'Address',
             'Temp', 'Date')

## 3 Data preparation

The data is now cleaned of missing values except the ones we left which will be used in the data preparation part to create others variables.

### 3.1 One-hot encoding

#### 3.1.1 `Burrito`

Let's display some burrito names:

In [50]:
df.select('Burrito').distinct().show(5)

+------------------+
 Burrito|
+------------------+
 Colimas burrito|
 Shrimp|
 Azteca|
 Spicy a la Diabla|
Bitchin California|
+------------------+
only showing top 5 rows

In [51]:
#We delete spaces at the end of the word:
df = df.withColumn('Burrito', fn.trim(df.Burrito))

#We set it to lowercase:
df = df.withColumn('Burrito', fn.lower(df.Burrito))

#Showing some districs to check operations have been done correctly:
df.select('Burrito').distinct().show(5)

#Number of different burritos:
n_burrito = neighborhood = df.select('Burrito').distinct().count()
print(f'There are {n_burrito} different burritos.')

+------------------+
 Burrito|
+------------------+
 mauna lani|
 quesaburro|
monster california|
 local|
 deborah's special|
+------------------+
only showing top 5 rows

There are 116 different burritos.

Let's display how many times a Burrito appears in the dataset:

In [53]:
count_burrito = df.groupBy('Burrito').agg(fn.count('Location').alias('count')).orderBy(("count"), ascending=False)
display(count_burrito)

Burrito,count
california,127
carne asada,35
carnitas,24
california everything,16
local,14
surf & turf,14
al pastor,13
adobada,10
surfin california,9
custom,5


The California burrito in the most represented one. Let's show the burritos which appears more than 5 times:

In [55]:
known_burrito = count_burrito.where(count_burrito['count'] > 5).select('Burrito')
display(known_burrito)

Burrito
california
carne asada
carnitas
california everything
surf & turf
local
al pastor
adobada
surfin california


We can now create a dummy variable for each of these burrito's name:

In [57]:
for burrito in [str(row.Burrito) for row in known_burrito.collect()]:
  dum = fn.when(df['Burrito']==burrito, 1).otherwise(0)
  df = df.withColumn(burrito, dum) #We apply the decision rule
  df = df.withColumnRenamed(burrito,burrito.replace(" ", "_")) #Replace spaces by underscores

We noticed the Burrito named "Surf and Turf" is also named "Surf & Turf":

In [59]:
df = df.withColumnRenamed('surf_&_turf', 'surf_and_turf') #We rename the first column created
dum_surfturf = fn.when(df['Burrito'] == 'surf and turf', 1).otherwise(df['surf_and_turf']) #to get all obs. who ate this burrito
df = df.withColumn('surf_and_turf', dum_surfturf) #we apply the decision rule

In [60]:
## For later on for data analaysis
df = df.withColumn('burrito_forstat', fn.when(df['california']+df['carne_asada']+df['carnitas']+df['local']+df['surf_and_turf']+
                                              df['california_everything']+df['adobada']+
                                              df['al_pastor']+df['surfin_california'] + df['surf_and_turf'] ==1, df.Burrito)\
                                         .otherwise("Special Burrito"))

Observations where none of these dummies are set at 1 are 'special tacos', meaning there are creation from the restaurant or not famous/ more original tacos. There is no dummies to represent these tacos to avoid identification issue for the classification.

Now we can drop the `Burrito` column:

In [62]:
df = df.drop('Burrito')

#### 3.1.2 `Neighborhood`

Let's count the number of different locations and neighborhoods, and do some text cleaning on it:

In [65]:
#We delete spaces at the end of the word:
df = df.withColumn('Neighborhood', fn.trim(df.Neighborhood))

#We set it to lowercase:
df = df.withColumn('Neighborhood', fn.lower(df.Neighborhood))

#Showing some districs to check operations have been done correctly:
df.select('Neighborhood').distinct().show(5)

#Number of different neighborhoods:
n_neighborhood = df.select('Neighborhood').distinct().count()
print(f'There are {n_neighborhood} different neighborhoods.')

+------------+
Neighborhood|
+------------+
 san marcos|
 carlsbad|
 ocean beach|
 new york|
 la jolla|
+------------+
only showing top 5 rows

There are 39 different neighborhoods.

In some cases the neighborhood is missing, we will reshape the `Neighborhood` variable which will display 'No neighborhood' when this one is missing.

In [67]:
df.select('Neighborhood').distinct().show()

+-----------------+
 Neighborhood|
+-----------------+
 san marcos|
 carlsbad|
 ocean beach|
 new york|
 la jolla|
 pacific beach|
 university city|
 santa monica|
 del mar|
 mission beach|
 null|
 north park|
 south park|
la jolla village|
 linda vista|
 sorrento valley|
 carmel valley|
 webster|
 utc|
 hillcrest|
+-----------------+
only showing top 20 rows

In [68]:
count_neighborhood = df.groupBy('Neighborhood').agg(fn.count('Location').alias('count')).orderBy(("count"), ascending=False)
display(count_neighborhood)

Neighborhood,count
null,63
miramar,42
utc,42
north park,31
clairemont,29
linda vista,27
la jolla,23
encinitas,23
carlsbad,14
oceanside,14


In [69]:
known_neighborhood = count_neighborhood.where(count_neighborhood['count'] > 5).select('Neighborhood')
display(known_neighborhood)

Neighborhood
null
utc
miramar
north park
clairemont
linda vista
encinitas
la jolla
carlsbad
oceanside


In [70]:
for neigh in [str(row.Neighborhood) for row in known_neighborhood.collect()]:
  dum = fn.when(df['Neighborhood']==neigh, 1).otherwise(0)
  df = df.withColumn(neigh, dum)
  df = df.withColumnRenamed(neigh,neigh.replace(" ", "_"))
df.select('utc').show(10)

+---+
utc|
+---+
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
+---+
only showing top 10 rows

#### 3.1.3 `Location`

Let's do the same procedure for `Location`:

In [73]:
#We delete spaces at the end of the word:
df = df.withColumn('Location', fn.trim(df.Location))

#We set it to lowercase:
df = df.withColumn('Location', fn.lower(df.Location))

#Showing some districs to check operations have been done correctly:
df.select('Location').distinct().show(5)

#Number of different neighborhoods:
n_location = df.select('Location').distinct().count()
print(f'There are {n_location} different location.')

+--------------------+
 Location|
+--------------------+
 goody's|
caliente mexican ...|
 tacos por favor|
 king burrito|
 lolita's taco shop|
+--------------------+
only showing top 5 rows

There are 99 different location.

In [74]:
count_location = df.groupBy('Location').agg(fn.count('Location').alias('count')).orderBy(("count"), ascending=False)
display(count_location)

Location,count
taco villa,29
lucha libre north park,28
california burritos,27
rigoberto's taco shop,25
taco stand,25
los tacos,14
lolita's taco shop,13
vallarta express,13
los primos mexican food,12
el zarape,10


In [75]:
known_location = count_location.where(count_location['count'] > 5).select('Location')
display(known_location)

Location
taco villa
lucha libre north park
california burritos
rigoberto's taco shop
taco stand
los tacos
vallarta express
lolita's taco shop
los primos mexican food
el zarape


In [76]:
for location in [str(row.Location) for row in known_location.collect()]:
  dum = fn.when(df['Location']==location, 1).otherwise(0)
  df = df.withColumn(location, dum)
  df = df.withColumnRenamed(location,location.replace(" ", "_").replace("'", "_"))

#### 3.1.3 `Reviewer`

In [78]:
df = df.withColumn("Reviewer", fn.trim(df.Reviewer))
df = df.withColumn("Reviewer", fn.lower(df.Reviewer))
avg_reviewer =df.groupBy('Reviewer').agg(fn.mean(df.overall).alias('avg_grade'),
                                         fn.count(df.id).alias("n"))
print('There is ' + str(avg_reviewer.count()) + ' reviewers in the dataset')
avg_reviewer.show(5)

There is 104 reviewers in the dataset
+---------+-----------------+---+
 Reviewer| avg_grade| n|
+---------+-----------------+---+
 bankole|3.033333333333333| 3|
 brad| 4.5| 1|
 kevin| 2.0| 1|
 katie| 3.7| 2|
alejandro| 4.2| 1|
+---------+-----------------+---+
only showing top 5 rows

In [79]:
alpha=5
meanrev = df.agg({"overall" : 'avg'}).first()[0]
smoothed = fn.udf(lambda x,n: (n*x + meanrev*alpha)/(n + alpha))
avg_reviewer = avg_reviewer.withColumn("avg_smooth", smoothed(avg_reviewer.avg_grade, avg_reviewer.n))
avg_reviewer.show(5)

+---------+-----------------+---+------------------+
 Reviewer| avg_grade| n| avg_smooth|
+---------+-----------------+---+------------------+
 bankole|3.033333333333333| 3| 3.400554230997625|
 brad| 4.5| 1|3.7674056413301664|
 kevin| 2.0| 1| 3.3507389746635|
 katie| 3.7| 2|3.6434905497115713|
alejandro| 4.2| 1|3.7174056413301666|
+---------+-----------------+---+------------------+
only showing top 5 rows

In [80]:
mean = avg_reviewer.agg({"avg_smooth" : 'avg'}).first()[0]
stdev = avg_reviewer.agg({"avg_smooth" : 'std'}).first()[0]
correction = fn.udf(lambda x: (x-mean))
avg_reviewer = avg_reviewer.withColumn("avg_grade_scaled",correction(avg_reviewer.avg_smooth))
avg_reviewer.show(5)  

+---------+-----------------+---+------------------+--------------------+
 Reviewer| avg_grade| n| avg_smooth| avg_grade_scaled|
+---------+-----------------+---+------------------+--------------------+
 bankole|3.033333333333333| 3| 3.400554230997625| -0.2481877648084989|
 brad| 4.5| 1|3.7674056413301664| 0.1186636455240424|
 kevin| 2.0| 1| 3.3507389746635| -0.2980030211426241|
 katie| 3.7| 2|3.6434905497115713|-0.00525144609455...|
alejandro| 4.2| 1|3.7174056413301666| 0.06866364552404258|
+---------+-----------------+---+------------------+--------------------+
only showing top 5 rows

In [81]:
avg_reviewer = avg_reviewer.drop("avg_grade")
avg_reviewer = avg_reviewer.drop("avg_smooth")
avg_reviewer = avg_reviewer.drop("n")

### 3.2 Discretization

- We will discretize continuous variables according to quartiles because some of them contain too much NAs, hence a class will be dedicated to them.
- In order to avoid collinearity we will only keep the NA class, the first and second quartile.

#### 3.2.1 `Length`

To get an idea of the distribution of the variable `Length`:

In [86]:
print('Minimum: ', df.select(fn.min(df['Length'])).collect()[0][0])
print('First quartile: ', df.where(df['Length'].isNotNull()).approxQuantile('Length', [0.25], 0.25)[0])
print('Median: ',df.where(df['Length'].isNotNull()).approxQuantile('Length', [0.5], 0.25)[0])
print('Third quartile: ',df.where(df['Length'].isNotNull()).approxQuantile('Length', [0.75], 0.25)[0])
print('Maximum: ',df.select(fn.max(df['Length'])).collect()[0][0])

Minimum: 15.0
First quartile: 15.0
Median: 22.0
Third quartile: 26.0
Maximum: 26.0

We can create dummys according to the quartiles and missing values:

In [88]:
#NA:
length_na = fn.when(df['Length'].isNull(), 1).otherwise(0)
df = df.withColumn('length_na', length_na)

#1st quartile:
length_25 = fn.when(df['Length'] <= 15, 1).otherwise(0)
df = df.withColumn('length_25', length_25)

#2nd quartile:
length_50 = fn.when( (df['Length'] > 15) & (df['Length'] < 22), 1).otherwise(0)
df = df.withColumn('length_50', length_50)

#### 3.2.2 `Circum`

To get an idea of the distribution of the variable `Circum`:

In [91]:
print("Minimum: ", df.select(fn.min(df['Circum'])).collect()[0][0])
print("First quartile: ", df.where(df['Circum'].isNotNull()).approxQuantile("Circum", [0.25], 0.25)[0])
print("Median: ", df.where(df['Circum'].isNotNull()).approxQuantile("Circum", [0.5], 0.25)[0])
print("Third quartile: ", df.where(df['Circum'].isNotNull()).approxQuantile("Circum", [0.75], 0.25)[0])
print("Maximum: ", df.select(fn.max(df['Circum'])).collect()[0][0])

Minimum: 17.0
First quartile: 17.0
Median: 22.5
Third quartile: 29.0
Maximum: 29.0

We can create dummys according to the quartiles and missing values:

In [93]:
#NA:
circum_na = fn.when(df['Circum'].isNull(), 1).otherwise(0)
df = df.withColumn('circum_na', circum_na)

#1st quartile:
circum_25 = fn.when(df['Circum'] <= 17, 1).otherwise(0)
df = df.withColumn('circum_25', circum_25)

#2nd quartile:
circum_50 = fn.when( (df['Circum'] > 17) & (df['Circum'] < 22.5), 1).otherwise(0)
df = df.withColumn('circum_50', circum_50)

#### 3.2.3 `Volume`

To get an idea of the distribution of the variable `Volume`:

In [96]:
print('Minimum: ', df.select(fn.min(df['Volume'])).collect()[0][0])
print('First quartile: ', df.where(df['Volume'].isNotNull()).approxQuantile('Volume', [0.25], 0.25)[0])
print('Median: ', df.where(df['Volume'].isNotNull()).approxQuantile('Volume', [0.5], 0.25)[0])
print('Third quartile: ', df.where(df['Volume'].isNotNull()).approxQuantile('Volume', [0.75], 0.25)[0])
print('Maximum: ', df.select(fn.max(df['Volume'])).collect()[0][0])

Minimum: 0.4
First quartile: 0.4
Median: 0.9
Third quartile: 1.54
Maximum: 1.54

We can create dummys according to the quartiles and missing values:

In [98]:
#NA:
volume_na = fn.when(df['Volume'].isNull(), 1).otherwise(0)
df = df.withColumn('volume_na', volume_na)

#1st quartile:
volume_25 = fn.when(df['Volume'] <= 0.4, 1).otherwise(0)
df = df.withColumn('volume_25', volume_25)

#2nd quartile:
volume_50 = fn.when( (df['Volume'] > 0.4) & (df['Volume'] < 0.79), 1).otherwise(0)
df = df.withColumn('volume_50', volume_50)

#### 3.2.4 `Yelp`

In [100]:
print("Minimum: ", df.select(fn.min(df['Yelp'])).collect()[0][0])
print("First quartile: ", df.where(df['Yelp'].isNotNull()).approxQuantile("Yelp", [0.25], 0.25)[0])
print("Median: ", df.where(df['Yelp'].isNotNull()).approxQuantile("Yelp", [0.5], 0.25)[0])
print("Third quartile: ", df.where(df['Yelp'].isNotNull()).approxQuantile("Yelp", [0.75], 0.25)[0])
print("Maximum: ", df.select(fn.max(df['Yelp'])).collect()[0][0])

Minimum: 2.5
First quartile: 2.5
Median: 4.0
Third quartile: 4.5
Maximum: 4.5

In [101]:
#NA:
yelp_na = fn.when(df['Yelp'].isNull(), 1).otherwise(0)
df = df.withColumn('yelp_na', yelp_na)

#1st quartile:
yelp_25 = fn.when(df['Yelp'] <= 2.5, 1).otherwise(0)
df = df.withColumn('yelp_25', yelp_25)

#2nd quartile:
yelp_50 = fn.when( (df['Yelp'] > 2.5) & (df['Yelp'] < 4.2), 1).otherwise(0)
df = df.withColumn('yelp_50', yelp_50)

#### 3.2.5 `Google`

In [103]:
print("Minimum: ", df.select(fn.min(df['Google'])).collect()[0][0])
print("First quartile: ", df.where(df['Google'].isNotNull()).approxQuantile("Google", [0.25], 0.25)[0])
print("Median: ", df.where(df['Google'].isNotNull()).approxQuantile("Google", [0.5], 0.25)[0])
print("Third quartile: ", df.where(df['Google'].isNotNull()).approxQuantile("Google", [0.75], 0.25)[0])
print("Maximum: ", df.select(fn.max(df['Google'])).collect()[0][0])

Minimum: 2.9
First quartile: 2.9
Median: 4.2
Third quartile: 5.0
Maximum: 5.0

In [104]:
#NA:
google_na = fn.when(df['Google'].isNull(), 1).otherwise(0)
df = df.withColumn('google_na', google_na)

#1st quartile:
google_25 = fn.when(df['Google'] <= 2.9, 1).otherwise(0)
df = df.withColumn('google_25', google_25)

#2nd quartile:
google_50 = fn.when( (df['Google'] > 4.3) & (df['Google'] < 5), 1).otherwise(0)
df = df.withColumn('google_50', google_50)

### 3.3 Sentiment Analysis / Opinion Mining

You can find below some example of the text on which we are going to execute sentiment analysis:

In [107]:
df.select('Notes').show(10)

+--------------------+
 Notes|
+--------------------+
 Way too small|
 Neutral|
 Neutral|
Everything is in ...|
Burritos cut in h...|
Not much bacon fl...|
 Bad shrimp|
Had been made too...|
tortilla became a...|
 Neutral|
+--------------------+
only showing top 10 rows

The output of the analyzer comes from the Vader package, it outputs scores about how much the notes is positive, negative or neutral. There are called *pos, neg* and *neu* respectively.

$$ pos, neg, neu  \in \[0,1\]  $$ 

It also provides a compound score of it: $$ compound \in [-1,1] $$

**NB:** we keep the "id" column values in the rdds because it will be used as schema further in order to comeback to a dataframe format.

In [110]:
#Sentiment analyzer
analyser = SentimentIntensityAnalyzer()

#Get sentiment scores: negativity, positivity, neutrality and compound 
def get_sentiment(x):
    return analyser.polarity_scores(x)

#Storing "Notes" as rdd
#lambda function: rank 0 for the id, 1 for the Notes
notes_rdd = df.select('id', 'Notes').rdd.map(lambda x : (x[0], x[1]))

#We map the function get_sentiment over the rdd we created:
#lambda function: rank 0 to keep the id, apply get_sentiment on rank 1
sentiment_rdd = notes_rdd.map(lambda x: (x[0],get_sentiment(x[1])))
sentiment_rdd.take(5)

Out[310]: [(120, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}),
 (121, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}),
 (122, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}),
 (222, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}),
 (223, {'neg': 0.144, 'neu': 0.685, 'pos': 0.171, 'compound': 0.1027})]

Now we can extract positivity, negativity and compound and set them into an rdd. We do not use neutrality as:

$$neg + neu + post = 1$$

We want to avoid for collinearity between columns.

In [112]:
#Reshaping the rdd
sentiment_rdd = sentiment_rdd.map(lambda x: (x[0], x[1]['neg'], x[1]['pos'], x[1]['compound']))

sentiment_rdd.take(5)

Out[311]: [(120, 0.0, 0.0, 0.0),
 (121, 0.0, 0.0, 0.0),
 (122, 0.0, 0.0, 0.0),
 (222, 0.0, 0.0, 0.0),
 (223, 0.144, 0.171, 0.1027)]

Here is a first example, when the comment is missing:

In [114]:
#we take the second individual for the sake of the example
i = 2
print(notes_rdd.take(i)[i-1][1])
print('Negativity:', sentiment_rdd.take(i)[i-1][1])
print('Positivity:', sentiment_rdd.take(i)[i-1][2])
print('Compound:', sentiment_rdd.take(i)[i-1][3])

Neutral
Negativity: 0.0
Positivity: 0.0
Compound: 0.0

As we can, the value we set for missing values outputs neither negativity nor positivity. Indeed it output neutrality at level 1.

Let's show another example:

In [116]:
i = 5
print(notes_rdd.take(i)[i-1][1])
print('Negativity:', sentiment_rdd.take(i)[i-1][1])
print('Positivity:', sentiment_rdd.take(i)[i-1][2])
print('Compound:', sentiment_rdd.take(i)[i-1][3])

Burritos cut in half, felt like a wrap instead of a burrito
Negativity: 0.144
Positivity: 0.171
Compound: 0.1027

The example above demonstrates pretty how much complexity can emanate from a note.

Checking sentiment_rdd is at the right size:

In [118]:
#Check number of rows:
assert sentiment_rdd.count() == df.count()

No error should come from the code above.

Let's create a dataframe containing sentiment information we collected:

In [120]:
sentiment_df = sentiment_rdd.toDF(['id', 'neg', 'pos', 'compound'])
sentiment_df.show(10)

+---+-----+-----+--------+
 id| neg| pos|compound|
+---+-----+-----+--------+
120| 0.0| 0.0| 0.0|
121| 0.0| 0.0| 0.0|
122| 0.0| 0.0| 0.0|
222| 0.0| 0.0| 0.0|
223|0.144|0.171| 0.1027|
400| 0.0| 0.0| 0.0|
401|0.778| 0.0| -0.5423|
402| 0.0| 0.0| 0.0|
403| 0.0| 0.0| 0.0|
362| 0.0| 0.0| 0.0|
+---+-----+-----+--------+
only showing top 10 rows

We can now join sentiment_df with the original raw data df:

In [122]:
#Join:
df = df.join(sentiment_df, 'id', 'left')

### 3.4 Term Frequency-Inverse Document Frequency (TF-IDF)

In [124]:
#We filter our analysis on real comments:
notes = df.where(df['Notes']!='Neutral').select("id", "Notes")

Firstly, we need to tokenize the comments:

In [126]:
tokenizer = Tokenizer(inputCol ='Notes', outputCol='words_token')
notes = tokenizer.transform(notes).select('id', 'words_token')
notes.show(5)

+---+--------------------+
 id| words_token|
+---+--------------------+
 26|[beans, are, surp...|
 29|[promising, start...|
 65|[a, little, small...|
191|[repeatedly, get,...|
222|[everything, is, ...|
+---+--------------------+
only showing top 5 rows

Secondly, we need to delete stopwords which don't give any interesting informations within the comment:

In [128]:
remover = StopWordsRemover(inputCol='words_token', outputCol='words_clean')
notes = remover.transform(notes).select('id', 'words_clean')
notes.show(5)

+---+--------------------+
 id| words_clean|
+---+--------------------+
 26|[beans, surprisin...|
 29|[promising, start...|
 65|[little, small., ...|
191|[repeatedly, get,...|
222|[everything, span...|
+---+--------------------+
only showing top 5 rows

In [129]:
notes_byword = notes.select(fn.explode(notes.words_clean).alias('word'), 'id')
notes_byword.show(30)

+------------+---+
 word| id|
+------------+---+
 beans| 26|
surprisingly| 26|
 good| 26|
 promising| 29|
 start| 29|
 finished| 29|
 poorly| 29|
 little| 65|
 small.| 65|
 rice| 65|
 beans| 65|
 cheese| 65|
 sour| 65|
 cream| 65|
 repeatedly|191|
 get|191|
 large|191|
 bites|191|
 sour|191|
 cream|191|
 everything|222|
 spanish.|222|
 get|222|
 taco|222|
 sope|222|
 started|293|
 really|293|
 great|293|
 ended|293|
 poor|293|
+------------+---+
only showing top 30 rows

Here, we delete the ponctuation to be able to recognize each word:

In [131]:
notes_byword = notes_byword.select(fn.regexp_replace(notes_byword.word, r'[?.!,;]', '').alias('word'), 'id')
notes_byword.show(10)

+------------+---+
 word| id|
+------------+---+
 beans| 26|
surprisingly| 26|
 good| 26|
 promising| 29|
 start| 29|
 finished| 29|
 poorly| 29|
 little| 65|
 small| 65|
 rice| 65|
+------------+---+
only showing top 10 rows

Here, we group words to get the frequency within the comment and the frequency of comment where the word appears in it.

In [133]:
tf_word = notes_byword.groupBy('id', 'word').agg(fn.count('id').alias('tf'))
df_word = notes_byword.groupBy('word').agg(fn.countDistinct('id').alias('df'))
nb_word = notes_byword.groupBy('id').agg(fn.count('word').alias('nb'))
tf_word.show(5)
df_word.show(5)
nb_word.show(5)
print('The total number of words is of '+ str(df_word.count()) + '.')

+---+------------+---+
 id| word| tf|
+---+------------+---+
 26| beans| 1|
 26|surprisingly| 1|
 26| good| 1|
 29| promising| 1|
 29| start| 1|
+---+------------+---+
only showing top 5 rows

+-----+---+
 word| df|
+-----+---+
 tons| 1|
 slow| 1|
thick| 1|
 got| 3|
 meat| 14|
+-----+---+
only showing top 5 rows

+---+---+
 id| nb|
+---+---+
 26| 3|
 29| 4|
 65| 7|
191| 6|
222| 5|
+---+---+
only showing top 5 rows

The total number of words is of 338.

In [134]:
df_word.orderBy(("df"), ascending=False).show(100)

+------------+---+
 word| df|
+------------+---+
 good| 41|
 burrito| 23|
 fries| 14|
 meat| 14|
 salsa| 13|
 really| 12|
 guac| 12|
 better| 10|
 tortilla| 10|
 small| 10|
 much| 9|
 like| 8|
 best| 8|
 cheese| 8|
 chips| 7|
 great| 7|
 little| 7|
 get| 6|
 time| 6|
 sauce| 6|
 cream| 6|
 sour| 6|
 way| 6|
 ingredients| 5|
 bad| 5|
 horchata| 5|
 rice| 5|
 spicy| 5|
 shrimp| 5|
 hot| 5|
 flavor| 5|
 cold| 5|
 dry| 4|
 beans| 4|
 overall| 4|
 bit| 4|
 probably| 4|
 first| 4|
 cooked| 3|
 amount| 3|
 enough| 3|
 onions| 3|
 verde| 3|
surprisingly| 3|
 plain| 3|
 carnitas| 3|
 one| 3|
 chipotle| 3|
 free| 3|
 got| 3|
 well| 3|
 terrible| 3|
 bland| 3|
 pretty| 3|
 size| 3|
 burritos| 3|
 big| 3|
 half| 3|
 food| 3|
 green| 3|
 quite| 2|
 taste| 2|
 tasted| 2|
 melted| 2|
 early| 2|
 past| 2|
 eating| 2|
 fresca| 2|
 rating| 2|
 eggs| 2|
 all| 2|
 made| 2|
 leftover| 2|
 usual| 2|
 enjoyed| 2|
 bottom| 2|
 quality| 2|
 chicken| 2|
 lime| 2|
 feels| 2|
 ordered| 2|
 uniformity| 2|
 add| 2|
 regular| 2|
 enchilada| 2|
 carrots| 2|
 cut| 2|
 super| 2|
 also| 2|
 instead| 2|
 far| 2|
 breakfast| 2|
 greasy| 2|
 fries:| 2|
 balance| 2|
 gone| 2|
 wrap| 2|
 bite| 2|
 feel| 2|
 grilled| 2|
+------------+---+
only showing top 100 rows

In [135]:
n_max = notes.count()/2
n_min = 10 
df_word = df_word.where(df_word.df<n_max)
df_word = df_word.where(df_word.df>=n_min)
df_word.show()
tf_word = tf_word.join(df_word,'word', 'inner')
tf_word = tf_word.drop('df')
tf_word.show(10)

+--------+---+
 word| df|
+--------+---+
 meat| 14|
 guac| 12|
 fries| 14|
 really| 12|
 salsa| 13|
 better| 10|
 small| 10|
tortilla| 10|
 good| 41|
 burrito| 23|
+--------+---+

+----+---+---+
word| id| tf|
+----+---+---+
meat|241| 1|
meat|190| 1|
meat|420| 1|
meat| 95| 1|
meat| 1| 1|
meat| 10| 1|
meat|218| 1|
meat| 61| 1|
meat|390| 1|
meat| 80| 1|
+----+---+---+
only showing top 10 rows

There are only 10 words left and they doen't seem informative by themselve. Combinaison of them can be interesting for analysis ( as 'not good') but alone it is only words about ingredients or feelings, the sentiment analyzer already captured this part and we also information about food inside the burrito. So we decided not to use TF-IDF.

Hence we can now drop the variable `Notes`:

In [137]:
#Drop:
df = df.drop('Notes')

### 3.4 Normalization of continuous variables

In order to treat data properly, the prediction model performs better with normalized data. Hence we will use the StandardScaler normalizer to do so, on a given variable \\(var\\), its mean \\(\overline{var}\\) and its standard deviation \\(\mathrm{std}(var)\\), we are going to normalize each observation as followed:

$$ \forall i \in \\{1,...,N\\}: scaled\\_var_i = \dfrac{var_i - \overline{var}}{\mathrm{std}(var)}$$

In [140]:
continuous_var = ['Cost', 'Hunger', 'neg', 'pos', 'compound']

In [141]:
for var in continuous_var: 
  mean = df.agg({var : 'avg'}).first()[0]
  stdev = df.agg({var : 'std'}).first()[0]
  stdz = fn.udf(lambda x: float((x-mean)/stdev))
  df = df.withColumn(var,stdz(df[var]))
  df = df.withColumn(var, fn.bround(var))

In [142]:
df.select(continuous_var).show(5)

+----+------+---+---+--------+
Cost|Hunger|neg|pos|compound|
+----+------+---+---+--------+
 0.0| 0.0|0.0|4.0| 2.0|
 0.0| 1.0|0.0|1.0| 1.0|
 1.0| -2.0|2.0|0.0| -1.0|
-1.0| 1.0|0.0|0.0| 0.0|
-1.0| -3.0|0.0|0.0| 0.0|
+----+------+---+---+--------+
only showing top 5 rows

### 3.5 Reshaping the target variable: `overall`

A logical decision would be to create a dummy which decision is ruled whether the target variable is above or below its median, let's compute the median:

In [145]:
#Computing the median with the approxQuantile function
#1st term: variable | 2nd term : quantile | 3rd term : relative error
median_overall = df.approxQuantile('overall', [0.5], 0.25)[0]

print('The median is about', median_overall, 'for the target variable.')

The median is about 4.0 for the target variable.

The target variable `overall` is originally continuous, we will reshape it into a dummy variable called `good_grade` according to the following decision rule:
$$
\overline{overall} = \displaystyle \dfrac{1}{n} \sum_{i=1}^{N} overall_i
$$
  
$$ \forall i \in \\{1,\ldots,N\\} ~: goodgrade_i = \begin{cases} 1 ~ \mathrm{if} ~ overall_i > \overline{overall}
\cr 0 ~ \mathrm{otherwise} \end{cases} $$  

Firstly, we shall compute \\(\overline{overall}\\) :

In [147]:
#Calculating the mean and round it to 2 decimals:
mean_overall = np.round(df.agg({'overall' : 'avg'}).first()[0],2)
print(mean_overall)

3.62

Now we are able to apply the decision rule:

In [149]:
#Create the decision rule; according to the overall grade, we will input the variable good_grade to 1 if it is higher than the mean, 0 otherwise
good_grade = fn.when(df['overall'] > mean_overall, 1)\
               .otherwise(0)

#We apply the decision rule and inpute it to actual column:
df = df.withColumn('good_grade', good_grade)

In [150]:
#To check the decision rule has been applied
print(df.select('overall', 'good_grade').show(10))

#Dropping the original target variable:
df_withall = df
df = df.drop('overall')

+-------+----------+
overall|good_grade|
+-------+----------+
 3.0| 0|
 3.0| 0|
 3.5| 0|
 3.0| 0|
 3.5| 0|
 2.5| 0|
 3.0| 0|
 3.8| 1|
 4.2| 1|
 3.9| 1|
+-------+----------+
only showing top 10 rows

None

## 4 Data Analysis

### 4.1 Descriptive statistics

Let's define the following function, it will compute the following statistics for a variable \\(var\\) according to the group: \\(goodgrade = k ~, k \in \\{0,1\\}\\)

1- \\(\mathrm{count}(goodgrade = k) = |i: goodgrade_i = k|\\)

2- \\(\mathrm{mean}(var) = \overline{var} = \displaystyle \dfrac{1}{N} \sum_{i = 1}^{N} var_i\\)

3- \\(\mathrm{std}(var) = \sqrt{\displaystyle \dfrac{1}{N} \sum_{i = 1}^{N} (var_i - \overline{var})^2}\\)

4- \\( \mathrm{min}(var)\\)

5- \\( \mathrm{quantile}(var)_p, p \in \\{0.25, 0.50, 0.75\\} = inf\\{var \in \mathbb{R}, p \le F(var)\\} ~ \mathrm{and} ~ F_X(x) = \mathbb{P}(X \le x) = p \\)

6- \\(\mathrm{max}(var) \\)

In [154]:
def descriptive(var):
  print(f'Statistics for the variable {var}:')
  df.groupby("good_grade")\
    .agg(fn.count(var).alias('count'),
                     fn.round(fn.mean(var),2).alias('mean'),
                     fn.round(fn.stddev(var),2).alias('std'),
                     fn.round(fn.min(var),2).alias('min'),
                     fn.round(fn.expr(f'percentile({var}, array(0.25))')[0],2).alias('%25'), 
                     fn.round(fn.expr(f'percentile({var}, array(0.5))')[0],2).alias('%50'),
                     fn.round(fn.expr(f'percentile({var}, array(0.75))')[0],2).alias('%75'),
                     fn.round(fn.max(var),2).alias('max'))\
    .show()

Let's run it on the following continuous features:

In [156]:
#list:
continuous_var = ['Length', 'Circum', 'Volume', 'Cost', 'Google', 'Yelp', 'neg', 'pos', 'compound']

#Run
for var in continuous_var:
  descriptive(var)

Statistics for the variable Length:
+----------+-----+-----+----+----+----+----+----+----+
good_grade|count| mean| std| min| %25| %50| %75| max|
+----------+-----+-----+----+----+----+----+----+----+
 1| 161|20.23|2.01|15.5|19.0|20.0|22.0|25.5|
 0| 122|19.78|2.16|15.0|18.5|19.5|21.0|26.0|
+----------+-----+-----+----+----+----+----+----+----+

Statistics for the variable Circum:
+----------+-----+-----+----+----+----+----+----+----+
good_grade|count| mean| std| min| %25| %50| %75| max|
+----------+-----+-----+----+----+----+----+----+----+
 1| 160|22.29|1.62|17.0|21.5|22.0|23.0|29.0|
 0| 121|21.94|1.96|17.0|20.5|22.0|23.0|29.0|
+----------+-----+-----+----+----+----+----+----+----+

Statistics for the variable Volume:
+----------+-----+----+----+----+----+----+----+----+
good_grade|count|mean| std| min| %25| %50| %75| max|
+----------+-----+----+----+----+----+----+----+----+
 1| 160| 0.8|0.14|0.41| 0.7|0.79|0.89|1.54|
 0| 121|0.76|0.16| 0.4|0.65|0.75|0.87|1.54|
+----------+-----+----+----+----+----+----+----+----+

Statistics for the variable Cost:
+----------+-----+----+----+----+----+---+---+----+
good_grade|count|mean| std| min| %25|%50|%75| max|
+----------+-----+----+----+----+----+---+---+----+
 1| 235|0.09|1.19|-3.0|-1.0|0.0|1.0|12.0|
 0| 186|-0.1|0.86|-2.0|-1.0|0.0|0.0| 3.0|
+----------+-----+----+----+----+----+---+---+----+

Statistics for the variable Google:
+----------+-----+----+----+---+---+---+---+---+
good_grade|count|mean| std|min|%25|%50|%75|max|
+----------+-----+----+----+---+---+---+---+---+
 1| 186| 4.2|0.35|3.3|4.0|4.3|4.4|5.0|
 0| 169| 4.1|0.37|2.9|3.9|4.1|4.4|5.0|
+----------+-----+----+----+---+---+---+---+---+

Statistics for the variable Yelp:
+----------+-----+----+----+---+---+---+---+---+
good_grade|count|mean| std|min|%25|%50|%75|max|
+----------+-----+----+----+---+---+---+---+---+
 1| 186|3.94|0.46|2.5|3.5|4.0|4.5|4.5|
 0| 169|3.73|0.47|2.5|3.5|3.5|4.0|4.5|
+----------+-----+----+----+---+---+---+---+---+

Statistics for the variable neg:
+----------+-----+----+----+---+---+---+---+---+
good_grade|count|mean| std|min|%25|%50|%75|max|
+----------+-----+----+----+---+---+---+---+---+
 1| 235|0.15|0.89|0.0|0.0|0.0|0.0|9.0|
 0| 186|0.31| 1.0|0.0|0.0|0.0|0.0|7.0|
+----------+-----+----+----+---+---+---+---+---+

Statistics for the variable pos:
+----------+-----+----+----+---+---+---+---+---+
good_grade|count|mean| std|min|%25|%50|%75|max|
+----------+-----+----+----+---+---+---+---+---+
 1| 235| 0.3|0.75|0.0|0.0|0.0|0.0|3.0|
 0| 186| 0.4|0.99|0.0|0.0|0.0|0.0|5.0|
+----------+-----+----+----+---+---+---+---+---+

Statistics for the variable compound:
+----------+-----+----+----+----+---+---+---+---+
good_grade|count|mean| std| min|%25|%50|%75|max|
+----------+-----+----+----+----+---+---+---+---+
 1| 235|0.26|0.92|-3.0|0.0|0.0|0.0|4.0|
 0| 186|0.15| 1.0|-3.0|0.0|0.0|0.0|3.0|
+----------+-----+----+----+----+---+---+---+---+

md Counterintuitively, the mean of the variable `pos` is higher for the \\( goodgrade = 0 \\) class, however the standard deviation relatively high so we cannot assert anything from those results.

We can now drop some of the continuous variables as we already created categories based on their quartiles and because the analysis is finished:

In [158]:
df = df.drop('Length', 'Circum', 'Volume', 'Google', 'Yelp')

We can also display the proportion of observations such that \\(goodgrade = 1 \\) per neighborhood as well as the number of different locations:

In [160]:
neighbh = df.groupBy(df.Neighborhood)\
            .agg(fn.mean(df.good_grade).alias('Proportion_of_good_grades'),
                 fn.count(df.id).alias('Number_of_tacos_evaluated'),
                 fn.countDistinct(df.Location).alias("Number_of_restaurant_concerned"))\
            .orderBy(("Proportion_of_good_grades"), ascending=False)
display(neighbh)

Neighborhood,Proportion_of_good_grades,Number_of_tacos_evaluated,Number_of_restaurant_concerned
bonita,1.0,2,1
ocean beach,1.0,4,1
la jolla village,1.0,1,1
san diego,1.0,1,1
temecula,1.0,4,1
cardiff,1.0,1,1
downtown,0.9,10,2
linda vista,0.8888888888888888,27,2
null,0.746031746031746,63,17
kearny mesa,0.7142857142857143,7,2


Besides, it is also possible to do it per reviewer:

In [162]:
reviwr = df.groupBy(df.Reviewer)\
           .agg(fn.mean(df.good_grade).alias("Proportion_of_good_grades"),
                fn.count(df.id).alias("Number_of_tacos_evaluated"))\
           .orderBy(("Proportion_of_good_grades"), ascending=False)

display(reviwr)

Reviewer,Proportion_of_good_grades,Number_of_tacos_evaluated
justin k,1.0,1
null,1.0,1
simon,1.0,2
chuck k,1.0,1
chris form,1.0,1
sarah,1.0,1
kalen,1.0,1
sankeerth,1.0,1
han mee,1.0,1
brian,1.0,2


We can now drop those variables:

In [164]:
df = df.drop('Neighborhood', 'Reviewer', 'Location')

### 4.2 Visual analysis

Let's focus our analysis on the type of burrito bought by customers, depending on they have a good grade (\\(goodgrade = 1 \\)) or not (\\(goodgrade = 0 \\)) :

In [167]:
#Counting the number of burrito per type for the good grade class:
good_gradeDF = df.where(df.good_grade==1)\
                 .groupBy(df.burrito_forstat)\
                 .agg(fn.count('id').alias('nb_burrito'))\
                 .select('nb_burrito')\
                 .orderBy(("burrito_forstat"), ascending=True)
#Setting it in a list:
list_L1 = [int(i.nb_burrito) for i in good_gradeDF.collect()]

#Counting the number of burrito per type for the good grade class:
bad_gradeDF = df.where(df.good_grade==0)\
                .groupBy(df.burrito_forstat)\
                .agg(fn.count('id').alias('nb_burrito'))\
                .select('nb_burrito')\
                .orderBy(("burrito_forstat"), ascending=True)
#Setting it in a list:
list_L0 = [int(i.nb_burrito) for i in bad_gradeDF.collect()]

#Create the different modalities
modalities = df.select('burrito_forstat').distinct().orderBy(("burrito_forstat"), ascending=True)
modalities =  [str(i.burrito_forstat) for i in modalities.collect()]

Let's display on a bar plot those figures:

In [169]:
#Required list for the totals:
totals = [i+j for i,j in zip(list_L1,list_L0)]
list_L1p = [i / j * 100 for i,j in zip(list_L1, totals)]
list_L0p = [i / j * 100 for i,j in zip(list_L0, totals)]
p=range(len(list_L1)) 

#Plot options:
barWidth = 0.7
plt.figure(figsize=(12,5))
plt.bar(p, list_L1p, color='#C7CF00', edgecolor='white', width=barWidth)
plt.bar(p, list_L0p, bottom=list_L1p, color='crimson', edgecolor='white', width=barWidth)
plt.xlabel('Type of burrito')
plt.ylabel('Proportion of good and bad grades')
plt.xticks(p, modalities, fontsize='small', size=6)
plt.gca().legend(('Good','Bad'))
display(plt.show())

We can do the exact same analysis based on the food inside the burritos, let's pick some of them: `beef`, `chicken`, `shrimp`, `fish`.

In [171]:
#Lists for the good grades:
list_beef = [int(i.nb) for i in df.where(df.Beef==1).select('good_grade').agg(fn.sum('good_grade').alias('nb')).collect()]
list_chicken = [int(i.nb) for i in df.where(df.Chicken==1).select('good_grade').agg(fn.sum('good_grade').alias('nb')).collect()]
list_shrimp = [int(i.nb) for i in df.where(df.Shrimp==1).select('good_grade').agg(fn.sum('good_grade').alias('nb')).collect()]
list_fish = [int(i.nb) for i in df.where(df.Fish==1).select('good_grade').agg(fn.sum('good_grade').alias('nb')).collect()]
#Gathering them all:
list_L1 = [list_beef[0], list_chicken[0], list_shrimp[0], list_fish[0]]

#Lists for the bad grades:
list_beef0 = [int(i.nb) for i in df.where(df.Beef==1).where(df.good_grade==0).agg(fn.count('id').alias('nb')).collect()]
list_chicken0 = [int(i.nb) for i in df.where(df.Chicken==1).where(df.good_grade==0).agg(fn.count('id').alias('nb')).collect()]
list_shrimp0 = [int(i.nb) for i in df.where(df.Shrimp==1).where(df.good_grade==0).agg(fn.count('id').alias('nb')).collect()]
list_fish0 = [int(i.nb) for i in df.where(df.Fish==1).where(df.good_grade==0).agg(fn.count('id').alias('nb')).collect()]
#Gathering them all:
list_L0 = [list_beef0[0], list_chicken0[0], list_shrimp0[0], list_fish0[0]]

#Name of the modalities:
modalities = ['Beef', 'Chicken', 'Shrimp', 'Fish']

In [172]:
#Creating the totals:
totals = [i+j for i,j in zip(list_L1,list_L0)]
list_L1p = [i / j * 100 for i,j in zip(list_L1, totals)]
list_L0p = [i / j * 100 for i,j in zip(list_L0, totals)]
p=range(len(list_L1)) 

#Plot options:
barWidth = 0.7
plt.figure(figsize=(12,5))
plt.bar(p, list_L1p, color='#C7CF00', edgecolor='white', width=barWidth)
plt.bar(p, list_L0p, bottom=list_L1p, color='crimson', edgecolor='white', width=barWidth)
plt.xlabel("Food inside the burrito")
plt.ylabel('Proportion of good and bad grades')
plt.xticks(p, modalities, fontsize='small', size=12)
plt.gca().legend(('Good','Bad'))
display(plt.show())

We can drop the variable `burrito_forstat`:

In [174]:
df = df.drop('burrito_forstat')

In [175]:
display(df)

id,Chips,Cost,Hunger,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,california,carne_asada,carnitas,california_everything,local,surf_and_turf,al_pastor,adobada,surfin_california,None,utc,miramar,north_park,clairemont,linda_vista,la_jolla,encinitas,carlsbad,oceanside,hillcrest,san_marcos,pacific_beach,downtown,university_heights,kearny_mesa,carmel_valley,taco_villa,lucha_libre_north_park,california_burritos,rigoberto_s_taco_shop,taco_stand,los_tacos,lolita_s_taco_shop,vallarta_express,los_primos_mexican_food,el_zarape,valentine_s_mexican_food,valentines_mexican_food,tony_s_fresh_mexican_food,cancun_mexican_&_seafood,lupe_s_taco_shop,length_na,length_25,length_50,circum_na,circum_25,circum_50,volume_na,volume_25,volume_50,yelp_na,yelp_25,yelp_50,google_na,google_25,google_50,neg,pos,compound,good_grade
26,0,0.0,0.0,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0.0,4.0,2.0,0
29,0,0.0,1.0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0.0,1.0,1.0,0
65,0,1.0,-2.0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,2.0,0.0,-1.0,0
191,1,-1.0,1.0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0.0,0.0,0.0,0
418,0,-1.0,-3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0.0,0.0,0.0,0
222,0,-1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0
270,0,-1.0,1.0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0.0,0.0,0.0,0
293,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,2.0,1.0,1.0,1
243,0,-1.0,1.0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0.0,0.0,0.0,1
278,0,-1.0,1.0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0.0,0.0,0.0,1


## 5 Modeling

In [177]:
df_path = '/FileStore/tables/Burrito_cleaned.csv'

#Loading the Burrito csv file
df = spark.read.format('csv').options(header='true', inferSchema='true').load(df_path)

#number of observations:
N = df.count()

In [178]:
display(df.limit(5))

id,Chips,Cost,Hunger,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,california,carne_asada,carnitas,california_everything,local,surf_and_turf,al_pastor,adobada,surfin_california,None,utc,miramar,north_park,clairemont,linda_vista,la_jolla,encinitas,carlsbad,oceanside,hillcrest,san_marcos,pacific_beach,downtown,university_heights,kearny_mesa,carmel_valley,taco_villa,lucha_libre_north_park,california_burritos,rigoberto_s_taco_shop,taco_stand,los_tacos,lolita_s_taco_shop,vallarta_express,los_primos_mexican_food,el_zarape,valentine_s_mexican_food,valentines_mexican_food,tony_s_fresh_mexican_food,cancun_mexican_&_seafood,lupe_s_taco_shop,length_na,length_25,length_50,circum_na,circum_25,circum_50,volume_na,volume_25,volume_50,yelp_na,yelp_25,yelp_50,google_na,google_25,google_50,neg,pos,compound,good_grade
26,0,0,0,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,4,2,0
29,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0
65,0,1,-2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,2,0,-1,0
191,1,-1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0
418,0,-1,-3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0


### 5.1 Last checks:

Checking for NAs:

In [181]:
na_per_column = df.select([fn.count(fn.when(df[c].isNull(), c)).alias(c) for c in df.columns])
display(na_per_column)

id,Chips,Cost,Hunger,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,california,carne_asada,carnitas,california_everything,local,surf_and_turf,al_pastor,adobada,surfin_california,None,utc,miramar,north_park,clairemont,linda_vista,la_jolla,encinitas,carlsbad,oceanside,hillcrest,san_marcos,pacific_beach,downtown,university_heights,kearny_mesa,carmel_valley,taco_villa,lucha_libre_north_park,california_burritos,rigoberto_s_taco_shop,taco_stand,los_tacos,lolita_s_taco_shop,vallarta_express,los_primos_mexican_food,el_zarape,valentine_s_mexican_food,valentines_mexican_food,tony_s_fresh_mexican_food,cancun_mexican_&_seafood,lupe_s_taco_shop,length_na,length_25,length_50,circum_na,circum_25,circum_50,volume_na,volume_25,volume_50,yelp_na,yelp_25,yelp_50,google_na,google_25,google_50,neg,pos,compound,good_grade
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


No missing or null values :)

In [183]:
assert df.count() == N

### 5.1 Feature selection

Let's compute correlations between features and the target variable:

In [186]:
for c in df.columns:
  matrix = df.corr(c,'good_grade' )
  print(c,":",np.round(matrix,2))

id : 0.11
Chips : -0.03
Cost : 0.09
Hunger : 0.12
Beef : -0.07
Pico : -0.11
Guac : -0.02
Cheese : -0.02
Fries : 0.0
Sour cream : -0.0
Pork : -0.08
Chicken : -0.1
Shrimp : -0.15
Fish : -0.05
Rice : -0.19
Beans : -0.2
Lettuce : -0.06
Tomato : -0.07
Bell peper : -0.07
Carrots : -0.05
Cabbage : -0.09
Sauce : -0.1
Salsa49 : -0.07
Cilantro : -0.06
Onion : -0.06
Taquito : 0.09
Pineapple : -0.07
Ham : -0.01
Chile relleno : 0.04
Nopales : -0.01
Lobster : -0.05
Queso : nan
Egg : 0.01
Mushroom : -0.04
Bacon : 0.02
Sushi : 0.06
Avocado : -0.12
Corn : -0.1
Zucchini : 0.04
california : 0.09
carne_asada : -0.06
carnitas : 0.03
california_everything : 0.15
local : 0.11
surf_and_turf : -0.11
al_pastor : -0.01
adobada : -0.05
surfin_california : -0.03
None : nan
utc : -0.09
miramar : 0.01
north_park : -0.12
clairemont : -0.08
linda_vista : 0.17
la_jolla : 0.02
encinitas : 0.05
carlsbad : -0.05
oceanside : -0.1
hillcrest : 0.04
san_marcos : -0.06
pacific_beach : 0.04
downtown : 0.11
university_heights : -0.02
kearny_mesa : 0.04
carmel_valley : -0.01
taco_villa : -0.0
lucha_libre_north_park : -0.13
california_burritos : 0.19
rigoberto_s_taco_shop : 0.1
taco_stand : 0.14
los_tacos : 0.14
lolita_s_taco_shop : -0.09
vallarta_express : -0.06
los_primos_mexican_food : -0.11
el_zarape : -0.02
valentine_s_mexican_food : 0.07
valentines_mexican_food : 0.13
tony_s_fresh_mexican_food : -0.03
cancun_mexican_&_seafood : 0.07
lupe_s_taco_shop : 0.03
length_na : -0.03
length_25 : -0.08
length_50 : -0.02
circum_na : -0.03
circum_25 : -0.01
circum_50 : -0.01
volume_na : -0.03
volume_25 : -0.05
volume_50 : -0.08
yelp_na : 0.16
yelp_25 : -0.05
yelp_50 : -0.2
google_na : 0.16
google_25 : -0.08
google_50 : 0.15
neg : -0.08
pos : -0.06
compound : 0.06
good_grade : 1.0

In [187]:
df = df.drop('Sour cream', 'Fries', 'Queso', 'None', 'taco_villa')

#### 5.1.1 Vector Assembler

First, we need to gather all features into one vector for each individual:

In [190]:
#Let's select only features:
features = df.drop('id', 'good_grade').columns

assembler = VectorAssembler(
    inputCols= features,
    outputCol='features')

df = assembler.transform(df)
display(df.select('id', 'features'))

id,features
26,"List(0, 92, List(3, 4, 5, 6, 12, 36, 51, 74, 77, 80, 88, 90, 91), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0, 2.0))"
29,"List(0, 92, List(2, 3, 11, 12, 18, 25, 46, 74, 77, 80, 85, 90, 91), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
65,"List(0, 92, List(1, 2, 3, 4, 5, 36, 63, 74, 77, 80, 83, 86, 89, 91), List(1.0, -2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, -1.0))"
191,"List(0, 92, List(0, 1, 2, 3, 6, 35, 54, 76, 79, 82, 85), List(1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
418,"List(0, 92, List(1, 2, 41, 69, 76, 79, 82, 83, 86), List(-1.0, -3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
222,"List(0, 92, List(1, 74, 77, 80), List(-1.0, 1.0, 1.0, 1.0))"
270,"List(0, 92, List(1, 2, 4, 5, 7, 41, 76, 79, 82, 83, 86), List(-1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
293,"List(0, 92, List(51, 76, 85, 89, 90, 91), List(1.0, 1.0, 1.0, 2.0, 1.0, 1.0))"
243,"List(0, 92, List(1, 2, 6, 28, 30, 45, 62, 79, 85, 88), List(-1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
278,"List(0, 92, List(1, 2, 3, 4, 5, 6, 35, 57, 68, 76, 79, 82, 85, 88), List(-1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"


The `features` column has the form: [0, number of columns, [columns containing non null values], [values of those columns]]

#### 5.1.2 Chi-Squared feature selection

Let's keep about 50% of the features we have at our disposal. For this we can set the percentile option within the ChiSqSelector method instead of the numTopfeatures option.

In [194]:
selector = ChiSqSelector(selectorType = "percentile", percentile = 0.5, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="good_grade")

df = selector.fit(df).transform(df)
display(df.select('id', 'features', 'selectedFeatures', 'good_grade').limit(5))

id,features,selectedFeatures,good_grade
26,"List(0, 92, List(3, 4, 5, 6, 12, 36, 51, 74, 77, 80, 88, 90, 91), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0, 2.0))","List(0, 46, List(0, 1, 6, 43, 45), List(1.0, 1.0, 1.0, 1.0, 4.0))",0
29,"List(0, 92, List(2, 3, 11, 12, 18, 25, 46, 74, 77, 80, 85, 90, 91), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 46, List(0, 5, 6, 11, 22, 40, 45), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",0
65,"List(0, 92, List(1, 2, 3, 4, 5, 36, 63, 74, 77, 80, 83, 86, 89, 91), List(1.0, -2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, -1.0))","List(0, 46, List(0, 1, 30, 39, 41, 44), List(1.0, 1.0, 1.0, 1.0, 1.0, 2.0))",0
191,"List(0, 92, List(0, 1, 2, 3, 6, 35, 54, 76, 79, 82, 85), List(1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 46, List(0, 17, 38, 40), List(1.0, 1.0, 1.0, 1.0))",0
418,"List(0, 92, List(1, 2, 41, 69, 76, 79, 82, 83, 86), List(-1.0, -3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 46, List(34, 38, 39, 41), List(1.0, 1.0, 1.0, 1.0))",0


Again, we can see Spark has its own way of treating sparse matrices, it does not store the zeros within the `features` and `selectedFeatures` columns.

In [196]:
df = df.drop('features')\
       .withColumnRenamed('selectedFeatures', 'features')\
       .withColumnRenamed('good_grade', 'label')\
       .select('features', 'label')

### 5.2 Splitting the dataset

We will split our dataset into a training set called `train` and a testing set called `test`, such that the model will be fitted on `train` which contains 80% of the individuals. The quality of our model will be measured on the test set (which contains the remaining 20% observations).

In [199]:
train, test = df.randomSplit([0.8, 0.2], seed=12345) #we set a seed for the reproduction of our results

Let's check the target variable is balanced across the variables:

In [201]:
def info_data(data):
  size = float(data.select('label').count())
  numPositives = data.select('label').where('label == 1').count()
  prop_ones = (float(numPositives)/float(size))*100
  print(f'The number of ones are {numPositives}/{data.count()}')
  print(f'Percentage of ones are {np.round(prop_ones,2)}%')

print('\n Train set:')
info_data(train)

print('\n Test set:')
info_data(test)

Train set:
The number of ones are 188/336
Percentage of ones are 55.95%

 Test set:
The number of ones are 47/85
Percentage of ones are 55.29%

As the proportion of \\(goodgrade = 1\\) approximates 50% within the training set, we can assert that this one is pretty balance regarding the distribution of the target variable.

### 5.4 Logistic Regression Model

Here is the loss function we aim to minimize in order to find the coefficients and predict as best as possible whether a customer likes the burrito or not:
$$
\mathcal{L}(w;x,y) = \mathrm{ln}(1+\mathrm{exp}(-yw^Tx)) ~ , ~ y = goodgrade
$$

After minimization of the loss, we end up with the estimated weights:
$$\widehat{w}$$

Then, we just need to set features values in order to predict a probability for an individual, applying the logistic function:

$$\forall i \in \\{1,...,N\\} ~ , ~ f(\widehat{w},x) = \dfrac{1}{1+e^{-\widehat{w}^Tx}}$$

However, the aim is to predict either 1 (the customer likes the burrito) or 0 (the customer dislikes the burrito). Then we have to compare the predicted probability to a specific threshold in order to assign either 1 or 0:

$$\forall i \in \\{1,...,N\\} ~ , ~ \forall threshold \in [0,1] ~, ~ \widehat{goodgrade_i} = \begin{cases} 1 ~ \mathrm{if} ~ f(\widehat{w}, x_i) > threshold \cr
0 ~ \mathrm{otherwise} \cr
\end{cases}$$

#### 5.4.1 Fitting the model

In [206]:
#Initialize the model,
lr = LogisticRegression(labelCol='label', featuresCol='features', maxIter=50)

#Fit on the train set:
logit_model = lr.fit(train)

# Extract the summary from the returned LogisticRegressionModel instance trained
# in the earlier example
trainingSummary = logit_model.summary

#Set the model threshold to maximize F-Measure
fMeasure = trainingSummary.fMeasureByThreshold
maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
    .select('threshold').head()['threshold']
lr.setThreshold(bestThreshold)

#Get predictions for both train and test sets:
predict_train_logit = logit_model.transform(train)
predict_test_logit = logit_model.transform(test)

#Show up 10 first rows:
predict_test_logit.select('label', 'prediction', 'probability').show(10)

+-----+----------+--------------------+
label|prediction| probability|
+-----+----------+--------------------+
 0| 0.0|[0.93080303006728...|
 0| 0.0|[0.92719760188720...|
 1| 0.0|[0.99999975490988...|
 0| 0.0|[0.72379870387097...|
 0| 1.0|[8.33633157714867...|
 0| 1.0|[0.47633232625442...|
 1| 1.0|[0.07417745319090...|
 1| 1.0|[0.04819218207358...|
 1| 1.0|[0.05667311730805...|
 1| 1.0|[0.46501660287855...|
+-----+----------+--------------------+
only showing top 10 rows

#### 5.4.2 Performance Evaluation

To measure the performance we have to compute those metrics:

-TP: customer who likes the burrito and who was predicted well

-TN: customer who dislikes the burrito and who was predicted well

-FP: customer who dislikes the burrito and who was predicted wrong

-FN: customer who likes the burrito and who was predicted wrong

-P = TP+FN :total number of positives

-N = TN+FP :total number of negatives

In order to get performance metrics about our predictions, we will store into a single **class** called `binary_metrics`, the following metrics:

-accuracy = \\(\dfrac{\mathrm{TP+TN}}{\mathrm{TP+TN+FP+FN}}\\)

-precision = \\(\dfrac{\mathrm{TP}}{\mathrm{TP+FP}}\\)

-recall = \\(\dfrac{\mathrm{TP}}{\mathrm{TP+FN}}\\)

-F1 Score = \\(2 \times \dfrac{\mathrm{precision} \times \mathrm{recall}}{\mathrm{precision} + \mathrm{recall}}\\)
 
-AUC (Area Under the Curve) = \\(\displaystyle \int_0^1 \dfrac{\mathrm{TP}}{\mathrm{P}} d(\dfrac{\mathrm{FP}}{\mathrm{N}}) \\)

In [209]:
class binary_metrics():
  
  def __init__(self, data):
    #Store data into self:
    self.data = data
    
    #Prediction metrics
    self.predictionAndLabels = data.select('prediction', 'label').rdd.map(lambda x : (float(x[0]), float(x[1])))
    self.metrics = MulticlassMetrics(self.predictionAndLabels)
    
    #ROC metrics:
    self.scoreAndLabels = data.select('probability', 'label').rdd.map(lambda x : (float(x[0][1]), float(x[1])))
    self.roc = BinaryClassificationMetrics(self.scoreAndLabels)
    
  def accuracy(self):
    accuracy = self.predictionAndLabels.filter(lambda x: x[0] == x[1]).count()/self.predictionAndLabels.count()
    return np.round(accuracy,2)
  
  def precision(self):
    precision = self.metrics.precision(label = 1.0)
    return np.round(precision,2)
  
  def recall(self):
    recall = self.metrics.recall(label = 1.0)
    return np.round(recall,2)
  
  def fmeasure(self):
    fmeasure = self.metrics.fMeasure(label = 1.0)
    return np.round(fmeasure,2)
  
  def confusion_matrix(self):
    confusion = pd.DataFrame(self.metrics.confusionMatrix().toArray())
    return confusion
  
  def AUC(self):
    auc = self.roc.areaUnderROC
    return np.round(auc,2)

In [210]:
#Get results from the class for predictions from the test set:
logit_test_results = binary_metrics(predict_test_logit)
logit_train_results = binary_metrics(predict_train_logit)

print('\n Train set:')
print('Confusion matrix: \n')
print(logit_train_results.confusion_matrix(), "\n")

print('Accuracy =',logit_train_results.accuracy())
print('Precision =',logit_train_results.precision())
print('Recall =',logit_train_results.recall())
print('F1 Score =',logit_train_results.fmeasure())
print('Area Under the Curve =',logit_train_results.AUC())


print('\n Test set:')
print('Confusion matrix: \n')
print(logit_test_results.confusion_matrix(), "\n")

print('Accuracy =',logit_test_results.accuracy())
print('Precision =',logit_test_results.precision())
print('Recall =',logit_test_results.recall())
print('F1 Score =',logit_test_results.fmeasure())
print('Area Under the Curve =',logit_test_results.AUC())

Train set:
Confusion matrix: 

 0 1
0 102.0 46.0
1 40.0 148.0 

Accuracy = 0.74
Precision = 0.76
Recall = 0.79
F1 Score = 0.77
Area Under the Curve = 0.83

 Test set:
Confusion matrix: 

 0 1
0 19.0 19.0
1 10.0 37.0 

Accuracy = 0.66
Precision = 0.66
Recall = 0.79
F1 Score = 0.72
Area Under the Curve = 0.69

Let's compute try another logistic model with K-Fold Cross Validation:

#### 5.4.3 K-Fold Cross Validation

In [213]:
#Let's set the parameter grid for hyper-parameter optimization:
paramGrid = ParamGridBuilder()\
    .addGrid(lr.aggregationDepth,[2,5,10])\
    .addGrid(lr.elasticNetParam,[0.5, 1.0])\
    .addGrid(lr.fitIntercept,[False, True])\
    .addGrid(lr.maxIter,[10, 50])\
    .addGrid(lr.regParam,[0, 0.5]) \
    .build()

evaluator= BinaryClassificationEvaluator(rawPredictionCol= 'rawPrediction', labelCol='label')

# Create 5-fold CrossValidator
cv_logit = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=4)

# Run cross validations
cvModel_logit = cv_logit.fit(train)

# this will likely take a fair amount of time because of the amount of models that we're creating and testing
predict_train_logit_k = cvModel_logit.transform(train)
predict_test_logit_k = cvModel_logit.transform(test)
print("The area under ROC for train set after CV  is {}".format(evaluator.evaluate(predict_train_logit_k)))
print("The area under ROC for test set after CV  is {}".format(evaluator.evaluate(predict_test_logit_k)))

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
The area under ROC for train set after CV is 0.8336328349626221
The area under ROC for test set after CV is 0.6875699888017914

Hyper-parameter optimization does not seem efficient as the AUC has not been affected.

### 5.5 Decision Tree classifier

First we have to set the label and features indexers to set the data in a good format for the decision tree model:

In [217]:
#Label indexer:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df)

#Feature indexer (maxCategories = 2, such that every column with categories > 2 will be treated as a continuous variable):
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(df)

In [218]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Train model. This also runs the indexers.
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

dt_model = pipeline.fit(train)

# Make predictions:
predict_train_dt = dt_model.transform(train)
predict_test_dt = dt_model.transform(test)

In [219]:
#Get results from the class for predictions from the test set:
dt_test_results = binary_metrics(predict_test_dt)
dt_train_results = binary_metrics(predict_train_dt)

print('\n Train set:')
print('Confusion matrix: \n')
print(dt_train_results.confusion_matrix(), "\n")

print('Accuracy =',dt_train_results.accuracy())
print('Precision =',dt_train_results.precision())
print('Recall =',dt_train_results.recall())
print('F1 Score =',dt_train_results.fmeasure())
print('Area Under the Curve =',dt_train_results.AUC())


print('\n Test set:')
print('Confusion matrix: \n')
print(dt_test_results.confusion_matrix(), "\n")

print('Accuracy =',dt_test_results.accuracy())
print('Precision =',dt_test_results.precision())
print('Recall =',dt_test_results.recall())
print('F1 Score =',dt_test_results.fmeasure())
print('Area Under the Curve =',dt_test_results.AUC())

Train set:
Confusion matrix: 

 0 1
0 29.0 119.0
1 119.0 69.0 

Accuracy = 0.29
Precision = 0.37
Recall = 0.37
F1 Score = 0.37
Area Under the Curve = 0.23

 Test set:
Confusion matrix: 

 0 1
0 17.0 21.0
1 27.0 20.0 

Accuracy = 0.44
Precision = 0.49
Recall = 0.43
F1 Score = 0.45
Area Under the Curve = 0.39

### 5.6 Random Forest classifier

In [221]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(df)

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
rf_model = pipeline.fit(train)

# Make predictions.
predict_train_rf = rf_model.transform(train)
predict_test_rf = rf_model.transform(test)

In [222]:
#Get results from the class for predictions from the test set:
rf_test_results = binary_metrics(predict_test_rf)
rf_train_results = binary_metrics(predict_train_rf)

print('\n Train set:')
print('Confusion matrix: \n')
print(rf_train_results.confusion_matrix(), "\n")

print('Accuracy =',rf_train_results.accuracy())
print('Precision =',rf_train_results.precision())
print('Recall =',rf_train_results.recall())
print('F1 Score =',rf_train_results.fmeasure())
print('Area Under the Curve =',rf_train_results.AUC())


print('\n Test set:')
print('Confusion matrix: \n')
print(rf_test_results.confusion_matrix(), "\n")

print('Accuracy =',rf_test_results.accuracy())
print('Precision =',rf_test_results.precision())
print('Recall =',rf_test_results.recall())
print('F1 Score =',rf_test_results.fmeasure())
print('Area Under the Curve =',rf_test_results.AUC())

Train set:
Confusion matrix: 

 0 1
0 92.0 56.0
1 174.0 14.0 

Accuracy = 0.32
Precision = 0.2
Recall = 0.07
F1 Score = 0.11
Area Under the Curve = 0.2

 Test set:
Confusion matrix: 

 0 1
0 27.0 11.0
1 43.0 4.0 

Accuracy = 0.36
Precision = 0.27
Recall = 0.09
F1 Score = 0.13
Area Under the Curve = 0.3

### 5.7 Gradient-boosted tree classifier

In [224]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df)
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(df)

# Train a GBT model.
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=10)

# Chain indexers and GBT in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])

# Train model.  This also runs the indexers.
gbt_model = pipeline.fit(train)

# Make predictions.
predict_train_gbt = gbt_model.transform(train)
predict_test_gbt = gbt_model.transform(test)


gbtModel = model.stages[2]
print(gbtModel)  # summary only

GBTClassificationModel (uid=GBTClassifier_9223abf10d50) with 10 trees

In [225]:
#Get results from the class for predictions from the test set:
gbt_test_results = binary_metrics(predict_test_gbt)
gbt_train_results = binary_metrics(predict_train_gbt)

print('\n Train set:')
print('Confusion matrix: \n')
print(gbt_train_results.confusion_matrix(), "\n")

print('Accuracy =',gbt_train_results.accuracy())
print('Precision =',gbt_train_results.precision())
print('Recall =',gbt_train_results.recall())
print('F1 Score =',gbt_train_results.fmeasure())
print('Area Under the Curve =',gbt_train_results.AUC())


print('\n Test set:')
print('Confusion matrix: \n')
print(gbt_test_results.confusion_matrix(), "\n")

print('Accuracy =',gbt_test_results.accuracy())
print('Precision =',gbt_test_results.precision())
print('Recall =',gbt_test_results.recall())
print('F1 Score =',gbt_test_results.fmeasure())
print('Area Under the Curve =',gbt_test_results.AUC())

Train set:
Confusion matrix: 

 0 1
0 35.0 113.0
1 150.0 38.0 

Accuracy = 0.22
Precision = 0.25
Recall = 0.2
F1 Score = 0.22
Area Under the Curve = 0.12

 Test set:
Confusion matrix: 

 0 1
0 18.0 20.0
1 35.0 12.0 

Accuracy = 0.35
Precision = 0.38
Recall = 0.26
F1 Score = 0.3
Area Under the Curve = 0.33

### 6. Determining the best burrito

In [227]:
df_path = '/FileStore/tables/df_withall.csv'

#Loading the Burrito csv file
df_withall = spark.read.format('csv').options(header='true', inferSchema='true').load(df_path)

In [228]:
display(df_withall)

id,Location,Chips,Cost,Hunger,Length,Circum,Volume,overall,Reviewer,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Neighborhood,Yelp,Google,california,carne_asada,carnitas,california_everything,local,surf_and_turf,al_pastor,adobada,surfin_california,burrito_forstat,None,utc,miramar,north_park,clairemont,linda_vista,la_jolla,encinitas,carlsbad,oceanside,hillcrest,san_marcos,pacific_beach,downtown,university_heights,kearny_mesa,carmel_valley,taco_villa,lucha_libre_north_park,california_burritos,rigoberto_s_taco_shop,taco_stand,los_tacos,lolita_s_taco_shop,vallarta_express,los_primos_mexican_food,el_zarape,valentine_s_mexican_food,valentines_mexican_food,tony_s_fresh_mexican_food,cancun_mexican_&_seafood,lupe_s_taco_shop,length_na,length_25,length_50,circum_na,circum_25,circum_50,volume_na,volume_25,volume_50,yelp_na,yelp_25,yelp_50,google_na,google_25,google_50,neg,pos,compound,good_grade
26,lola's 7 up market & deli,0,-0.0451011762890213,0.005724215453194856,null,null,null,3.0,scott,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,carlsbad,4.5,4.9,0,1,0,0,0,0,0,0,0,carne asada,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,-0.23685624780129297,3.847466564841965,2.315212563527023,0
29,colima's mexican food,0,-0.38643616257794405,1.241567654322525,null,null,null,3.0,eli,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,north park,4,4.2,0,0,0,0,0,0,0,0,0,Special Burrito,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,-0.23685624780129297,1.4584755888987595,0.6255548148108032,0
65,taco stand,0,0.550561838999491,-1.8480409428508004,null,null,null,3.5,richard,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,0,1,0,0,0,0,0,0,0,carne asada,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1.5395372921708859,-0.41943771557400883,-1.209812761687522,0
191,los cabos,1,-0.7210783059984567,0.6236459348878599,18.5,21,0.65,3.0,scott,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,san marcos,4,4.1,1,0,0,0,0,0,0,0,0,california,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,-0.23685624780129297,-0.41943771557400883,-0.256936345541834,0
418,valentine's mexican food,0,-0.7143854631300466,-3.0838843817201305,17,20.5,0.57,3.5,anon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,0,0,0,0,0,0,1,0,0,al pastor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,-0.23685624780129297,-0.41943771557400883,-0.256936345541834,0
222,tacos la bala,0,-1.3903625928394818,0.005724215453194856,null,null,null,2.5,scott,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,houston,4.5,4.1,0,0,0,0,0,0,0,0,0,Special Burrito,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,-0.23685624780129297,-0.41943771557400883,-0.256936345541834,0
270,donato's taco shop,0,-0.7210783059984567,0.6236459348878599,19.5,21.5,0.72,3.0,scott,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,0,0,0,0,0,0,1,0,0,al pastor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,-0.23685624780129297,-0.41943771557400883,-0.256936345541834,0
293,pollos maria,0,-0.05179401915743141,0.005724215453194856,20.5,22.5,0.83,3.8,scott,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,carlsbad,4,3.8,0,0,0,0,0,0,0,0,0,Special Burrito,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,2.3383182799436106,1.4644183525205088,0.7202250419008716,1
243,rigoberto's tac

Here as a grade depend to the reviewer that give it and his severerity, we decided to reshape it with the smooth average of the reviwer's grade. Here the aim is to take into account the average grade gave by the reviewer, but also to take into account the number of review that he gave. If the reviewer gave only 1 review of 2 the score of the review shouldn't be as important as one that gave 10 review of 2. Because for the one that gave only one review of 2 it can be that it was justified and a very bad burrito.

In [230]:
window_spec = Window.partitionBy('Reviewer')
avg_grade = fn.mean('overall').over(window_spec)
n = fn.count('id').over(window_spec)

df_withall= df_withall.withColumn('avg_grade', avg_grade)
df_withall= df_withall.withColumn('n', n)

alpha = 5
meanrevw = df_withall.agg({"overall" : 'avg'}).first()[0]
smoothed = fn.udf(lambda x,n: (n*x + meanrevw*alpha)/(n + alpha))
df_withall = df_withall.withColumn("avg_smooth", smoothed(df_withall.avg_grade, df_withall.n))

meanrevwr = df_withall.select('Reviewer', 'avg_smooth').distinct().agg(fn.mean('avg_smooth')).first()[0]
centr = fn.udf(lambda x: x-meanrevw)
df_withall = df_withall.withColumn("avg_grade_scaled",centr(df_withall.avg_smooth))
df_withall = df_withall.drop('avg_grade','n','avg_smooth')

df_withall = df_withall.withColumn('score', df_withall.overall -df_withall.avg_grade_scaled)

df_withall.select('id','Reviewer','score').show(5)


+---+--------+------------------+
 id|Reviewer| score|
+---+--------+------------------+
196| bankole|2.6203325385985754|
 61| bankole|3.2203325385985755|
 91| bankole|3.9203325385985757|
 51| brad| 4.353481128266033|
128| kevin|2.2701477949327002|
+---+--------+------------------+
only showing top 5 rows

In [231]:
compo = df_withall.columns[10:44]
compo.append('id')
compo.append('score')
df_bis = df_withall.select(compo)
display(df_bis)

Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa49,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,id,score
1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,196,2.6203325385985754
1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,61,3.2203325385985755
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,91,3.9203325385985757
1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51,4.353481128266033
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,128,2.2701477949327002
1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,167,4.103481128266034
1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,198,3.5773962198846294
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,92,3.777396219884629
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,418,3.5201477949327002
1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34,3.5345362198846293


In [232]:
#Delete variables that do not give any informations
to_drop = []
for var in df_bis.columns:
  if df_bis.agg({var : 'sum'}).first()[0]==0:
    to_drop.append(var)
print(to_drop)
df_bis.drop('Queso')

['Queso']
Out[384]: DataFrame[Beef: int, Pico: int, Guac: int, Cheese: int, Fries: int, Sour cream: int, Pork: int, Chicken: int, Shrimp: int, Fish: int, Rice: int, Beans: int, Lettuce: int, Tomato: int, Bell peper: int, Carrots: int, Cabbage: int, Sauce: int, Salsa49: int, Cilantro: int, Onion: int, Taquito: int, Pineapple: int, Ham: int, Chile relleno: int, Nopales: int, Lobster: int, Egg: int, Mushroom: int, Bacon: int, Sushi: int, Avocado: int, Corn: int, id: int, score: double]

In [233]:
ingredient = df_bis.columns[0:33]
for each in ingredient: 
  others = ingredient
  others.remove(each)
  for each2 in others:
    interact = fn.when(df_bis[each]== 0, 0).when(df_bis[each2]== 0, 0).otherwise(1)
    df_bis = df_bis.withColumn(str(each)+'_'+str(each2), interact)

In [234]:
features = df_bis.drop('id', 'score').columns

assembler = VectorAssembler(
    inputCols= features,
    outputCol='features')

df_bis = assembler.transform(df_bis)
display(df_bis.select('id', 'features'))


id,features
26,"List(0, 442, List(0, 1, 2, 3, 11, 34, 35, 36, 44, 66, 67, 75), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
29,"List(0, 442, List(0, 10, 11, 17, 24, 43, 44, 50, 57, 189, 195, 202, 357, 360), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
65,"List(0, 442, List(0, 1, 2, 34, 35, 66), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
191,"List(0, 442, List(0, 3, 4, 5, 36, 37, 38, 98, 99), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
418,"List(0, 442, List(), List())"
222,"List(0, 442, List(), List())"
270,"List(0, 442, List(1, 2, 6, 66, 70, 127), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
293,"List(0, 442, List(), List())"
243,"List(0, 442, List(3, 4, 28, 30, 98, 122, 124, 392, 406, 410), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"
278,"List(0, 442, List(0, 1, 2, 3, 4, 5, 34, 35, 36, 37, 38, 66, 67, 68, 69, 97, 98, 99), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))"


We have still too much information regarding the ingredient and combinaison of ingredients. Here, we have more variables than observations. So we decided to keep only the correlated to score ones as only master ingredients are interesting for us.

In [236]:
selector = ChiSqSelector(selectorType = "percentile", percentile = 0.2, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="score")

df_bis = selector.fit(df_bis).transform(df_bis)
display(df_bis.select('id', 'features', 'selectedFeatures', 'score').limit(5))

id,features,selectedFeatures,score
196,"List(0, 442, List(0, 1, 8, 10, 17, 32, 34, 41, 43, 50, 65, 156, 161, 168, 183, 184, 195, 210, 426, 434), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 88, List(1, 2, 11, 14, 15, 20, 42, 46, 47, 55, 57, 84, 86), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",2.6203325385985754
61,"List(0, 442, List(0, 3, 8, 10, 36, 41, 43, 157, 161, 185), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 88, List(1, 2, 14, 15, 42, 48), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",3.2203325385985755
91,"List(0, 442, List(0, 2, 3, 21, 35, 36, 54, 67, 85), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 88, List(7, 18, 25), List(1.0, 1.0, 1.0))",3.9203325385985757
51,"List(0, 442, List(0, 1, 2, 3, 4, 5, 34, 35, 36, 37, 38, 66, 67, 68, 69, 97, 98, 99), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 88, List(), List())",4.353481128266033
128,"List(0, 442, List(), List())","List(0, 88, List(), List())",2.2701477949327002


In [237]:
lr = LinearRegression(labelCol='score', featuresCol='selectedFeatures', maxIter=50)

#Fit on the train set:
linear_model = lr.fit(df_bis)


#Get predictions for both train and test sets:
prediction = linear_model.transform(df_bis)

#Show up 10 first rows:
prediction.select('SelectedFeatures','prediction').show(10)

+--------------------+------------------+
 SelectedFeatures| prediction|
+--------------------+------------------+
 (88,[],[])|3.7368493832779657|
(88,[2,15,55],[1....|3.2382577436597813|
 (88,[],[])|3.7368493832779657|
 (88,[],[])|3.7368493832779657|
 (88,[],[])|3.7368493832779657|
 (88,[],[])|3.7368493832779657|
 (88,[],[])|3.7368493832779657|
 (88,[],[])|3.7368493832779657|
(88,[9,10,33,34,8...| 3.833497044018666|
 (88,[],[])|3.7368493832779657|
+--------------------+------------------+
only showing top 10 rows

In [238]:
print(linear_model.coefficients)
print(linear_model.intercept)

[-0.7183382842060869,0.04189588420280072,-0.04351138972470366,-1.4645236369641823,-0.48750915137656275,-0.5266336413116088,-0.10899862723007565,0.05407695244462976,-0.13517030917799447,0.6468567278659326,0.2479456003934076,-0.04969425097386334,-0.12469525428490504,0.6377173068962949,0.051980737418739716,-0.264263097619709,-0.6122807271734254,-0.7568289922170315,0.05407695244462976,0.9051894335323829,-0.06945163194995448,-0.26433493073475267,0.23920591725388843,0.05705090588283555,0.16967689273794595,0.05407695244462976,0.9946647428105962,0.08104002107779984,0.955157486960698,-0.8266293365738392,-0.22076453126442966,0.9011918248160825,-0.10899862723007565,-1.0630470242830499,0.3100139749466664,0.24840098683389703,-0.06904223640044678,-0.3943131659095759,0.21259175355835994,-0.06969116941551273,-0.10899862723007565,-0.028840979497037882,0.019224873274214762,0.47371741839751125,1.09984591288727,0.05705090588283555,-0.06945163194995448,0.18595114197497697,-0.37743059987634986,0.7146859458244824,-0.15166772997463587,0.2765922269072684,-0.006335215613013031,-0.10899862723007565,-0.15166772997463587,-0.1908171522737719,-0.8736526295429042,-0.4720700160103663,0.38157592985494687,0.7117978016300589,0.9839331130400345,0.18402452963110355,0.3190188347784639,0.38157592985494687,0.38157592985494687,0.9231414361018264,0.18402452963110355,0.18402452963110355,-0.06969116941551273,-0.06969116941551273,0.057303116098512505,0.18402452963110355,0.15324865672208085,0.18402452963110355,-0.8736526295429042,0.38157592985494687,0.38157592985494687,-0.028840979497037882,0.4482174126181434,0.4482174126181434,-0.6030811935223306,-1.5104190236282782,0.3100139749466664,0.2479456003934076,-0.06945163194995448,0.18402452963110355,-0.04969425097386334,0.18402452963110355]
3.7368493832779657

In [239]:
print(linear_model.explainParams)

<bound method Params.explainParams of LinearRegression_aee108c8fba8>

In [240]:
coef = np.array([i for i in linear_model.coefficients])
coef_pos = np.where(coef > 0)
coef_pos[0]

Out[391]: array([ 1, 7, 9, 10, 13, 14, 18, 19, 22, 23, 24, 25, 26, 27, 28, 31, 34,
 35, 38, 42, 43, 44, 45, 47, 49, 51, 58, 59, 60, 61, 62, 63, 64, 65,
 66, 67, 70, 71, 72, 73, 75, 76, 78, 79, 82, 83, 85, 87])